<a href="https://colab.research.google.com/github/Nomiluks/Tutorials/blob/master/Sentiment_Analysis_Deep_Learning_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls "/content/drive/My Drive/data"

 Client_Labelled_Dataset.csv
 Dubai_RTA_Transport_Dataset.csv
 dubai_transport_2018_twitter.csv
 Dubai_Transport_Dataset_For_Labelling1.csv
'Dubai_Transport_Dataset_For_Labelling_English (1).xlsx'
 Dubai_Transport_Dataset_For_Labelling_English.csv
 Dubai_Transport_Dataset_For_Labelling_English.xlsx
 Dubai_Transport_Twitter_Dataset.csv
 External_Source_Dataset.csv
 indian_election.csv
 negative-words.txt
 oman_tourism_2018_tripadvisor.csv
 oman_tourism_2018_twitter.csv
 Political_Dataset_1.csv
 Political_Dataset_2.csv
 SentiWordNet_3.0.0_20130122.txt
 SentiWordNet_3.0.0.txt
 training.1600000.processed.noemoticon.csv
 Twitter_Sample_Dataset_1.csv
 Twitter_Sample_Dataset.csv
 Un_labeled_Dataset_from_Client.csv
 Unsupervised_Labeled_Dataset.csv


**Installing the pre-processing Libraries**

In [3]:
!pip install tweet-preprocessor
!pip install emoji
!pip install langdetect
!pip install detectlanguage
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Creating functions for the Cleaning of the Tweet/Text**

In [0]:
import string
import re
import preprocessor as p
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag
lemma = WordNetLemmatizer()
from bs4 import BeautifulSoup
from langdetect import detect_langs
import emoji
lemma = WordNetLemmatizer()
punctuation = list(string.punctuation)
from langdetect import DetectorFactory
DetectorFactory.seed = 0
stop  = stopwords.words('english') + punctuation + ['rt', 'via', 'with', 'new', 'get', 'it', 'go']
table = str.maketrans({key: None for key in string.punctuation})

p.set_options(p.OPT.URL, p.OPT.NUMBER)
def clean_function(x):
  x=emoji.demojize(x)
  x=p.clean(x)
  return x


def clean_tweet(x):
    x = BeautifulSoup(x, 'html.parser')
    x = x.get_text()   
    x = emoji.demojize(x)
    x = p.clean(x)
    x = x.replace('#','')
    x = x.replace('@','')
    x = [i for i in x.lower().split() if i not in stop]
    x = " ".join(x)
    x = re.sub(r"[^A-Z/a-z0-9(),!?\'\`.]", " ", x)
    x = re.sub((r"^[\W]*"), "", x)
    x = re.sub((r"\s[\W]\s"), ", ", x)
    x = [i for i in x.split() if len(i)>1]
    x = " ".join(x)
    x = x.translate(table) 

    normalized   = " ".join(lemma.lemmatize(word) for word in x.split())
    return normalized
  
  
from langdetect import detect
def language_detection_new(x):
  try:
    x=detect(x)
  except:
    x='unknown'
  return x

def get_tweet_label(x):
  return x.lower()

def model_output(x):
  return np.argmax(x)

**Mounting the Google drive to the Notebook**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Reading the Dataset from the Mount Point**

In [6]:
import pandas as pd
import numpy as np
df=pd.read_csv('/content/drive/My Drive/data/indian_election.csv', encoding='latin_1')
df.head(2)

,is_synced,politics_prob,u_bio,nonpolitics_prob,updated_at,u_website,p_picture,p_updated_at,p_shares,tweet_hashtags,p_created_at,u_longitude,u_profile_photo,u_posts,source,p_comments_text,sentiment_score,topic_id,u_fullname,p_likes,p_message,Label,query_hashtag,p_url,p_created_time,is_active,p_comments,is_link,is_validated,u_likes,u_date_joined,u_following,is_rechecked,u_verified,u_coordinates,created_at,p_id,p_is_shared,predicted_political_category,u_followers,u_location,query_string,predicted_sentiment_value,u_latitude,p_video_url,u_source
0,NaN,0.27,Public Policy & Political Affairs Analyst: Wit...,0.73,2019-05-07T09:34:13.625919,https://twitter.com/abhjoshi,NaN,2019-05-07T09:34:13.625919,0.0,#drought #farmers #agriculture #BhaJaPa #India...,2019-05-07T09:34:13.625919,78.667743,https://pbs.twimg.com/profile_images/104819996...,11304.0,Twitter,NaN,NaN,372,Abhishek Joshi,0.0,Banks are refusing loans to #drought hit #farm...,Negative,Indian Elections 2019,https://twitter.com/abhjoshi/status/1122826652...,2019-04-29T04:35:00,NaN,0.0,NaN,NaN,26.0,2011-10-07T04:40:00,119.0,NaN,False,NaN,2019-05-07T09:34:13.625919,1.12E+18,False,NOT,2228.0,INDIA,Indian election,NaN,22.351115,NaN,https://twitter.com/abhjoshi
1,NaN,0.32,Author~Political Observer~Always support Right...,0.68,2019-05-07T09:34:04.598634,https://www.instagram.com/AjaayGoel/,NaN,2019-05-07T09:34:04.598634,0.0,NaN,2019-05-07T09:34:04.598634,78.667743,https://pbs.twimg.com/profile_images/112060096...,35546.0,Twitter,NaN,NaN,372,Ajaay Goel,2.0,Why Modi and BJP so sure to win 2019 Elections...,Neutral,Indian Elections 2019,https://twitter.com/AjaayGoel24/status/1122827...,2019-04-29T04:36:00,NaN,1.0,NaN,NaN,59399.0,2009-06-13T08:21:00,4977.0,NaN,False,NaN,2019-05-07T09:34:04.598634,1.12E+18,False,NOT,184.0,India,Indian election,NaN,22.351115,NaN,https://twitter.com/AjaayGoel24


In [0]:
new_dataframe=pd.DataFrame()
new_dataframe['Tweet']=df['p_message']
new_dataframe['Label']=df['Label']
new_dataframe=new_dataframe.dropna()
new_dataframe['Language']=new_dataframe['Tweet'].apply(language_detection_new)
#Keeping only the English Tweets from the Dataset
new_dataframe=new_dataframe[new_dataframe.Language=='en']

new_dataframe['Clean_Text']=new_dataframe['Tweet'].apply(clean_tweet)

In [8]:
new_dataframe.head()

,Tweet,Label,Language,Clean_Text
0,Banks are refusing loans to #drought hit #farm...,Negative,en,bank refusing loan drought hit farmer priority...
1,Why Modi and BJP so sure to win 2019 Elections...,Neutral,en,modi bjp sure win election revelation
2,Caste your Vote or get out and make a Differen...,Positive,en,caste vote make difference indianelections2019...
3,And another one from @fazilkhan331 + @Sheikh_S...,Neutral,en,another one fazilkhan331 sheikh saaliq indiane...
4,More Pics: @iamsrk & @gaurikhan clicked with L...,Neutral,en,pic iamsrk gaurikhan clicked little abram cast...


In [9]:
new_dataframe.shape

(831, 4)

**Using Flair Word Embeddings **

In [0]:
!pip -q install flair

In [11]:
from flair.embeddings import WordEmbeddings

# downloading the English Twitter embeddings model
en_embedding = WordEmbeddings('en-twitter')

2019-05-19 18:49:34,422 this function is deprecated, use smart_open.open instead


In [0]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence
#converting the word embeddings for each sentence to document embedding using 'mean'
document_embeddings = DocumentPoolEmbeddings([en_embedding])

In [0]:
sentence = Sentence('The grass is green . And the sky is blue .')

document_embeddings.embed(sentence)

In [14]:
len(sentence.get_embedding())

100

In [0]:
def get_sentence_embedding(x):
  try:
    sentence=Sentence(x)
    document_embeddings.embed(sentence)
    return np.array(sentence.get_embedding())
  except:
    return np.zeros(100)
  

Sample Embeddings output

In [16]:
get_sentence_embedding('The grass is green . And the sky is blue .')

array([-9.93069112e-02, -1.34626910e-01,  2.18430027e-01,  2.45591819e-01,
       -1.41971812e-01,  4.12981629e-01, -2.28435084e-01, -1.79186389e-02,
       -2.39931449e-01, -7.20176399e-02,  7.55842775e-02,  4.91699297e-03,
       -4.25098133e+00, -1.98307261e-01,  2.12185159e-01,  9.30747315e-02,
        3.01299453e-01,  1.35724898e-02, -1.21419452e-01, -1.96116075e-01,
       -3.25855464e-01, -3.28485407e-02, -4.55643572e-02, -1.61668111e-04,
        3.11907738e-01, -4.48498160e-01, -2.69164741e-01,  1.26205459e-01,
        3.64723653e-01, -7.54852146e-02, -3.38018209e-01, -2.98062693e-02,
       -3.43292892e-01, -4.03872617e-02,  1.33023947e-01, -1.80230923e-02,
       -2.88112704e-02, -6.30979091e-02, -6.02829875e-03, -1.30605832e-01,
       -9.42751825e-01, -1.13012344e-01, -3.24660808e-01,  2.23492563e-01,
        6.30479097e-01, -3.95057052e-02,  2.81220347e-01,  1.83366001e-01,
        1.48922995e-01, -1.75709620e-01, -2.02483997e-01, -1.53428540e-01,
        1.57820001e-01, -

In [0]:
new_dataframe['Embedding']=new_dataframe['Clean_Text'].apply(get_sentence_embedding)

In [18]:
new_dataframe.head()

,Tweet,Label,Language,Clean_Text,Embedding
0,Banks are refusing loans to #drought hit #farm...,Negative,en,bank refusing loan drought hit farmer priority...,"[0.16026314, 0.0882285, 0.16123097, -0.1270487..."
1,Why Modi and BJP so sure to win 2019 Elections...,Neutral,en,modi bjp sure win election revelation,"[0.22818549, 0.8129783, 0.11859601, 0.10746799..."
2,Caste your Vote or get out and make a Differen...,Positive,en,caste vote make difference indianelections2019...,"[-0.089494504, 0.028712504, 0.119205, 0.198306..."
3,And another one from @fazilkhan331 + @Sheikh_S...,Neutral,en,another one fazilkhan331 sheikh saaliq indiane...,"[0.11204466, 0.0011766677, 0.11091683, 0.15985..."
4,More Pics: @iamsrk & @gaurikhan clicked with L...,Neutral,en,pic iamsrk gaurikhan clicked little abram cast...,"[0.15090166, 0.26583132, -0.07382625, 0.023092..."


Dataset Size Distribution

In [19]:
new_dataframe.groupby('Label').size()

Label
Negative     89
Neutral     567
Positive    175
dtype: int64

# Data Structuring for Neural Nets

In [20]:
from sklearn import preprocessing
from numpy import array
from sklearn.model_selection import train_test_split 
import keras

Using TensorFlow backend.


In [0]:
le = preprocessing.LabelEncoder()
X_train = np.stack(new_dataframe['Embedding'])

y_train = new_dataframe['Label'].values.tolist()
le.fit(y_train)
y_train=le.transform(y_train)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

# y_train = keras.utils.to_categorical(y_train, num_classes=3)
# y_test  = keras.utils.to_categorical(y_test, num_classes=3)

In [36]:
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((664, 100), (664,)), ((167, 100), (167,)))

# Class Balancing

In [50]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                   np.unique(y_train),
                                                   y_train)
class_weights

array([2.83760684, 0.4822077 , 1.74278215])

# **Fully-Connected Network**

In [62]:
import tensorflow as tf
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping


def generate_sample_weights(training_data, class_weight_dictionary): 
    sample_weights = [class_weight_dictionary[np.where(one_hot_row==1)[0][0]] for one_hot_row in training_data]
    return np.asarray(sample_weights)
  
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')>=0.85):
      print("Reached 85% accuracy so cancelling training!")
      self.model.stop_training = True
      
callbacks = myCallback()

model = Sequential()
model.add(Dense(512, input_dim=100))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print(model.summary())

history = model.fit(X_train, y_train,
          epochs=100,
          callbacks=[callbacks],
          shuffle=True,
          validation_split=0.3,
          class_weight = 'auto'
          # class_weight=class_weights
         )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 512)               51712     
_________________________________________________________________
batch_normalization_26 (Batc (None, 512)               2048      
_________________________________________________________________
activation_43 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 3)                 1539      
_________________________________________________________________
activation_44 (Activation)   (None, 3)                 0         
Total params: 55,299
Trainable params: 54,275
Non-trainable params: 1,024
_________________________________________________________________
No

In [63]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9127
Testing Accuracy:  0.7246


**Evaluation**

In [64]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print('Classification Report')
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(y_test, y_pred, target_names=target_names))

Confusion Matrix
[[ 2  9  0]
 [ 0 96 12]
 [ 0 25 23]]
Classification Report
              precision    recall  f1-score   support

    Negative       1.00      0.18      0.31        11
     Neutral       0.74      0.89      0.81       108
    Positive       0.66      0.48      0.55        48

   micro avg       0.72      0.72      0.72       167
   macro avg       0.80      0.52      0.56       167
weighted avg       0.73      0.72      0.70       167



# **Convolutional Neural Network**

In [0]:
# will do it in next phase